In [1]:
!pip install warmup_scheduler datasets
!git clone https://github.com/dome272/wuerstchen/
%cd wuerstchen
!pip install transformers git+https://github.com/pabloppp/pytorch-tools

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 24.5 MB/s eta 0:00:00
  Created wheel for warmup_scheduler: filename=warmup_scheduler-0.3-py3-none-any.whl size=2967 sha256=e8bc3c3d2c16fb91185adc4480163384220e9d444ce3b5b72e607f7e32f89bab
  Stored in directory: /root/.cache/pip/wheels/59/01/9e/d1820991c32916e9808c940f572b462f3e46427f3e76c4d852
Successfully built warmup_scheduler
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3
Cloning into 'wuerstchen'...
remote: Enumera

In [2]:
import os
import time
import torch
import torchvision
from torch import nn, optim
from torch.utils.data import DataLoader
from warmup_scheduler import GradualWarmupScheduler
from tqdm import tqdm
import numpy as np
import shutil
from transformers import AutoTokenizer, CLIPTextModel
from torchtools.utils import Diffuzz
from vqgan import VQModel
from modules import Paella, sample, EfficientNetEncoder, Prior
from utils import WebdatasetFilter, transforms, effnet_preprocess, identity
import transformers
from transformers.utils import is_torch_bf16_available, is_torch_tf32_available

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

!mkdir models
!wget -P models https://huggingface.co/dome272/wuerstchen/resolve/main/model_stage_b.pt
!wget https://huggingface.co/dome272/wuerstchen/resolve/main/model_stage_c_ema.pt
!wget -P models https://huggingface.co/dome272/wuerstchen/resolve/main/vqgan_f4_v1_500k.pt

--2024-05-10 22:23:49--  https://huggingface.co/dome272/wuerstchen/resolve/main/model_stage_b.pt
Resolving huggingface.co (huggingface.co)... 18.172.134.88, 18.172.134.24, 18.172.134.4, ...
Connecting to huggingface.co (huggingface.co)|18.172.134.88|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs.huggingface.co/repos/b9/6e/b96ebc9f17a640b8e89f419a072a1ef4e6dadbcdf69a668bf2518fc87cd52c63/6cfd09dbe8765a7f973dc96f7b1f2e8328e35bab414f556d67d994676c19c6d5?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27model_stage_b.pt%3B+filename%3D%22model_stage_b.pt%22%3B&Expires=1715639029&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcxNTYzOTAyOX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9yZXBvcy9iOS82ZS9iOTZlYmM5ZjE3YTY0MGI4ZTg5ZjQxOWEwNzJhMWVmNGU2ZGFkYmNkZjY5YTY2OGJmMjUxOGZjODdjZDUyYzYzLzZjZmQwOWRiZTg3NjVhN2Y5NzNkYzk2ZjdiMWYyZTgzMjhlMzViYWI0MTRmNTU2ZDY3ZDk5NDY3NmMxOWM2ZDU%7EcmV

In [5]:
clip_model = CLIPTextModel.from_pretrained("laion/CLIP-ViT-H-14-laion2B-s32B-b79K").to(
        device).eval().requires_grad_(False)
clip_tokenizer = AutoTokenizer.from_pretrained("laion/CLIP-ViT-H-14-laion2B-s32B-b79K")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/4.72k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/3.94G [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/904 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

In [6]:
transformers.utils.logging.set_verbosity_error()

# PARAMETERS
updates = 30000
warmup_updates = 10
ema_start = 5
ema_every = 1
ema_beta = 0.9
batch_size = 16
grad_accum_steps = 1
max_iters = updates * grad_accum_steps
print_every = grad_accum_steps
extra_ckpt_every = grad_accum_steps
lr = 1e-4

In [7]:
dataset_path = ""
run_name = "Würstchen-Prior-CLIP-Text-v2"
output_path = f"output/würstchen/{run_name}"
os.makedirs(output_path, exist_ok=True)
checkpoint_dir = f"models/würstchen/"
checkpoint_path = os.path.join(checkpoint_dir, run_name, "model.pt")
os.makedirs(os.path.join(checkpoint_dir, run_name), exist_ok=True)

In [8]:
from torch.utils.data import Dataset

class ImageCaptionDataset(Dataset):
    def __init__(self, dataset, transform=None):
        self.dataset = dataset
        self.transform = transform

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        item = self.dataset[idx]
        image = item['image']
        text = item['text']

        if self.transform:
            image = self.transform(image)

        return {'image': image, 'text': text}

# defining transformations from utils
transform = transforms

#loading in dataset
from datasets import load_dataset
hf_dataset = load_dataset("YaYaB/onepiece-blip-captions")['train']
image_caption_dataset = ImageCaptionDataset(hf_dataset, transform=transform)

from torch.utils.data import DataLoader
dataloader = DataLoader(image_caption_dataset, batch_size=16, shuffle=True)


Generating train split:   0%|          | 0/856 [00:00<?, ? examples/s]

In [9]:
image_caption_dataset.__getitem__(0)

{'image': tensor([[[0.0000, 0.0000, 0.0000,  ..., 0.7216, 0.7216, 0.7216],
          [0.0026, 0.0014, 0.0014,  ..., 0.7216, 0.7216, 0.7216],
          [0.0093, 0.0071, 0.0057,  ..., 0.7216, 0.7216, 0.7216],
          ...,
          [0.4997, 0.5034, 0.5054,  ..., 0.1686, 0.1686, 0.1686],
          [0.5137, 0.5146, 0.5150,  ..., 0.1686, 0.1686, 0.1686],
          [0.5276, 0.5276, 0.5264,  ..., 0.1686, 0.1686, 0.1686]],
 
         [[0.4794, 0.4784, 0.4759,  ..., 0.7137, 0.7137, 0.7137],
          [0.4850, 0.4831, 0.4822,  ..., 0.7137, 0.7137, 0.7137],
          [0.4917, 0.4895, 0.4881,  ..., 0.7137, 0.7137, 0.7137],
          ...,
          [0.7899, 0.7936, 0.7956,  ..., 0.1686, 0.1686, 0.1686],
          [0.8039, 0.8048, 0.8052,  ..., 0.1686, 0.1686, 0.1686],
          [0.8178, 0.8178, 0.8166,  ..., 0.1686, 0.1686, 0.1686]],
 
         [[0.9766, 0.9756, 0.9720,  ..., 0.7176, 0.7176, 0.7176],
          [0.9909, 0.9890, 0.9854,  ..., 0.7176, 0.7176, 0.7176],
          [0.9976, 0.9953, 0.99

In [10]:
# preparing models

vqmodel = VQModel().to(device)
vqmodel.load_state_dict(torch.load("models/vqgan_f4_v1_500k.pt", map_location=device)['state_dict'])
vqmodel.eval().requires_grad_(False)

diffuzz = Diffuzz(device=device)

clip_model = CLIPTextModel.from_pretrained("laion/CLIP-ViT-H-14-laion2B-s32B-b79K").to(
    device).eval().requires_grad_(False)
clip_tokenizer = AutoTokenizer.from_pretrained("laion/CLIP-ViT-H-14-laion2B-s32B-b79K")

pretrained_checkpoint = torch.load("models/model_stage_b.pt", map_location=device)
effnet = EfficientNetEncoder(effnet="efficientnet_v2_l").to(device)
effnet.load_state_dict(pretrained_checkpoint['effnet_state_dict'])
effnet.eval().requires_grad_(False)

generator = Paella(byt5_embd=1024).to(device)
generator.load_state_dict(pretrained_checkpoint['state_dict'])
generator.eval().requires_grad_(False)

del pretrained_checkpoint

model = Prior(c_in=16, c=1536, c_cond=1024, c_r=64, depth=32, nhead=24).to(device)
model_ema = Prior(c_in=16, c=1536, c_cond=1024, c_r=64, depth=32, nhead=24).to(device).eval().requires_grad_(False)


optimizer = optim.AdamW(model.parameters(), lr=lr)
scheduler = GradualWarmupScheduler(optimizer, multiplier=1, total_epoch=warmup_updates)
scaler = torch.cuda.amp.GradScaler()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Using EffNet L.


Downloading: "https://download.pytorch.org/models/efficientnet_v2_l-59c71312.pth" to /root/.cache/torch/hub/checkpoints/efficientnet_v2_l-59c71312.pth
100%|██████████| 455M/455M [00:02<00:00, 183MB/s]


In [12]:
checkpoint_path = "../drive/MyDrive/Applied_CV/Final_Project/v8_good/1000_iters_prior.pth"

checkpoint = torch.load(checkpoint_path, map_location=device)

model.load_state_dict(checkpoint['ema_state_dict'])
model.eval().requires_grad_(False)

model_ema.load_state_dict(checkpoint['ema_state_dict'])
model_ema.eval().requires_grad_(False)

Prior(
  (projection): Conv2d(16, 1536, kernel_size=(1, 1), stride=(1, 1))
  (cond_mapper): Sequential(
    (0): Linear(in_features=1024, out_features=1536, bias=True)
    (1): LeakyReLU(negative_slope=0.2)
    (2): Linear(in_features=1536, out_features=1536, bias=True)
  )
  (blocks): ModuleList(
    (0): ResBlock(
      (depthwise): Conv2d(1536, 1536, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=1536)
      (norm): LayerNorm2d((1536,), eps=1e-06, elementwise_affine=False)
      (channelwise): Sequential(
        (0): Linear(in_features=1536, out_features=6144, bias=True)
        (1): GELU(approximate='none')
        (2): GlobalResponseNorm()
        (3): Dropout(p=0.1, inplace=False)
        (4): Linear(in_features=6144, out_features=1536, bias=True)
      )
    )
    (1): TimestepBlock(
      (mapper): Linear(in_features=64, out_features=3072, bias=True)
    )
    (2): AttnBlock(
      (norm): LayerNorm2d((1536,), eps=1e-06, elementwise_affine=False)
      (attention): 

In [15]:
def train():
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    minLoss = float('inf')

    start_iter = 1001

    for it in range(start_iter, max_iters + 1):
        bls = time.time()
        batch = next(iter(dataloader))
        ble = time.time() - bls

        images = batch['image']
        captions = batch['text']

        for i in captions:
          i += " cartoon"

        images = images.to(device)

        with torch.no_grad():
            effnet_features = effnet(effnet_preprocess(images))
            with torch.cuda.amp.autocast():
                if np.random.rand() < 0.05:
                    clip_captions = [''] * len(captions)
                else:
                    clip_captions = captions
                clip_tokens = clip_tokenizer(clip_captions, truncation=True, padding="max_length",
                                             max_length=clip_tokenizer.model_max_length, return_tensors="pt").to(device)
                clip_text_embeddings = clip_model(**clip_tokens).last_hidden_state

                t = (1 - torch.rand(images.size(0), device=device)).mul(1.08).add(0.001).clamp(0.001, 1.0)
                noised_embeddings, noise = diffuzz.diffuse(effnet_features, t)

            pred_noise = model(noised_embeddings, t, clip_text_embeddings)
            loss = nn.functional.mse_loss(pred_noise, noise, reduction='none').mean(dim=[1, 2, 3])
            loss_adjusted = (loss * diffuzz.p2_weight(t)).mean() / grad_accum_steps

            loss_adjusted.requires_grad = True


        if it % grad_accum_steps == 0 or it == max_iters:
            loss_adjusted.backward()
            grad_norm = nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()
            scheduler.step()
            optimizer.zero_grad(set_to_none=True)
        else:
            loss_adjusted.backward()

        if it % ema_every == 0 or it == max_iters:
            if it < ema_start:
                model_ema.load_state_dict(model.state_dict())
            else:
                model_ema.update_weights_ema(model, beta=ema_beta)

        ema_loss = loss.mean().item()


        if it % 100 == 0:
            print(f"ITER {it} - loss {ema_loss}")

        if it % 1000 == 0:
            torch.save({
                'state_dict': model.state_dict(),
                'ema_state_dict': model_ema.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'scheduler_last_step': scheduler.last_epoch,
                'iter': it,
                'metrics': {
                    'ema_loss': ema_loss,
                },
                'grad_scaler_state_dict': scaler.state_dict(),
            }, f'../drive/MyDrive/Applied_CV/Final_Project/{it}_iters_prior.pth')

        if ema_loss < minLoss:
            torch.save({
                'state_dict': model.state_dict(),
                'ema_state_dict': model_ema.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'scheduler_last_step': scheduler.last_epoch,
                'iter': it,
                'metrics': {
                    'ema_loss': ema_loss,
                },
                'grad_scaler_state_dict': scaler.state_dict(),
            }, f'../drive/MyDrive/Applied_CV/Final_Project/{it}_iters_prior.pth')
            minLoss = ema_loss
            print("loss:", minLoss, "iteration:", it)

    print("Training finished.")

In [16]:
train()

loss: 0.6473814845085144 iteration: 1001
loss: 0.3789982497692108 iteration: 1002
loss: 0.3454592227935791 iteration: 1005
loss: 0.32943764328956604 iteration: 1043
ITER 1100 - loss 0.6123157739639282
loss: 0.31865936517715454 iteration: 1181
ITER 1200 - loss 0.43416184186935425
loss: 0.2790018916130066 iteration: 1267
ITER 1300 - loss 1.0714340209960938
ITER 1400 - loss 0.7164398431777954
ITER 1500 - loss 0.924060583114624
loss: 0.2730201184749603 iteration: 1515
loss: 0.19056741893291473 iteration: 1589
ITER 1600 - loss 0.7920637130737305
ITER 1700 - loss 0.7516186833381653
ITER 1800 - loss 0.7640020847320557
ITER 1900 - loss 0.7584080696105957
ITER 2000 - loss 0.6869649887084961
ITER 2100 - loss 0.62515789270401
loss: 0.18535417318344116 iteration: 2148
ITER 2200 - loss 0.6103413105010986
loss: 0.17497000098228455 iteration: 2252
ITER 2300 - loss 0.8104994893074036
ITER 2400 - loss 0.6999738812446594
ITER 2500 - loss 0.8235727548599243
ITER 2600 - loss 0.77801513671875
ITER 2700 - l

KeyboardInterrupt: 